In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array

In [ ]:
random_img = np.random.randint(len(train_data))
print(train_data.shape)
print(max(train_data[random_img]))
print(min(train_data[random_img]))

In [ ]:
#scale data to range [-1,1] --> as we use tanh activation in the output of the generated images. 
train_images =  train_data*2 - 1
print(max(train_images[random_img]))
print(min(train_images[random_img]))

In [ ]:
kernel_size = [5,5]
stride = [2,2]

def gen(input_tensor, reuse=False):#input tensor is z: [-1, 100]
    with tf.variable_scope('generator'):
        act_fun_g = tf.nn.relu

        fully = tf.layers.dense(input_tensor, 
                                7*7*64,
                                activation=act_fun_g,
                                reuse=reuse,
                                name='g_hid1')
        fully_r = tf.reshape(fully, shape=[-1,7,7,64])

        cnn_T_layer1 = tf.layers.conv2d_transpose(fully_r, 32, kernel_size, strides=stride,
                                                  padding='same',
                                                  activation=act_fun_g,
                                                  reuse=reuse,
                                                  name='g_CNN_T1')#image is now double the size --> 14x14
        
        cnn_T_layer2 = tf.layers.conv2d_transpose(cnn_T_layer1, 1, kernel_size, strides=stride,
                                                  padding='same',
                                                  activation=None,
                                                  reuse=reuse,
                                                  name='g_CNN_T2')#image is now double the size --> 28x28

        output = tf.reshape(cnn_T_layer2,[-1, 784])
        return tf.nn.tanh(output)

In [ ]:
def dis(input_tensor, reuse=False):#input_tensor is an image [-1, 784]
    with tf.variable_scope('discriminator'):
        act_fun_d = tf.nn.leaky_relu#default leak: alpha = 0.2

        input_tensor_r = tf.reshape(input_tensor, shape=[-1, 28, 28, 1])

        cnn_layer1 = tf.layers.conv2d(input_tensor_r, 32, kernel_size, strides=stride,
                                      padding='same',
                                      activation=act_fun_d,
                                      reuse=reuse,
                                      name='d_CNN1')#image is now half the size --> 14x14

        cnn_layer2 = tf.layers.conv2d(cnn_layer1, 64, kernel_size, strides=stride,
                                      padding='same',
                                      activation=act_fun_d,
                                      reuse=reuse,
                                      name='d_CNN2')#image is now half the size --> 7x7
        cnn_layer2_r = tf.reshape(cnn_layer2,[-1, 7*7*64])

        output = tf.layers.dense(cnn_layer2_r, 
                                 1,
                                 activation=None,
                                 reuse=reuse,
                                 name='d_out')

        return tf.nn.sigmoid(output)

In [ ]:
def gen_train(d_fake, LR):
    with tf.variable_scope('', reuse=True):
        global_step = tf.get_variable('global_step',dtype=tf.int32)
  
    g_loss = tf.reduce_mean(-tf.log(d_fake))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=LR,beta1=0.5)
    g_var = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')
    g_train = optimizer.minimize(g_loss, var_list=g_var, global_step=global_step)
    return g_train, g_loss

In [ ]:
def dis_train(d_fake, d_real, LR):
    d_loss = -0.5*tf.reduce_mean((tf.log(d_real) + tf.log(1.0 - d_fake)))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=LR,beta1=0.5)
    d_var = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
    d_train = optimizer.minimize(d_loss, var_list=d_var)
    return d_train, d_loss

In [ ]:
def random_batch(x, batch_size):
    loc = np.random.randint(len(x), size=batch_size)
    return x[loc,:]

In [ ]:
#run time!
z_dim = 100
LR = 0.0002
x_dim = 784
batch_size = 128

def training():
    with tf.Graph().as_default():
        sess = tf.Session()
        global_step = tf.get_variable('global_step', shape=[],dtype=tf.int32)
        
        x = tf.placeholder(tf.float32, [batch_size, x_dim], 'real-images')
        
        z = tf.random_normal([batch_size, z_dim], mean=0, stddev=1.0)#Latent: Gaussian
        
        gen_x = gen(z, reuse=False)
        
        d_fake = dis(gen_x, reuse=False)
        d_real = dis(x, reuse=True)
        
        g_train, g_loss = gen_train(d_fake, LR)
        d_train, d_loss = dis_train(d_fake, d_real, LR)
       
        initial_var = tf.global_variables_initializer()
        sess.run(initial_var)
        
        train = True
        step = 0
        
        while train == True:
            real_data = random_batch(train_images, batch_size)
            _g, _d = sess.run([g_train, d_train], feed_dict={x: real_data})
            
            if step % 100 == 0:
                loss_g, loss_d, gen_images = sess.run([g_loss, d_loss, gen_x], feed_dict={x: real_data})
                print()
                print('------------------------------------------------------------------------------')
                print('At step:', step)
                print('G loss =', loss_g)
                print('D loss =', loss_d)
                plt.subplot(1, 4, 1)
                plt.title('real')
                plt.imshow(np.reshape(real_data[0], [28,28]), cmap='gray')
                
                plt.subplot(1, 4, 2)
                plt.title('generated')
                plt.imshow(np.reshape(gen_images[0], [28,28]), cmap='gray')
                
                plt.subplot(1, 4, 3)
                plt.title('real')
                plt.imshow(np.reshape(real_data[1], [28,28]), cmap='gray')
                
                plt.subplot(1, 4, 4)
                plt.title('generated')
                plt.imshow(np.reshape(gen_images[1], [28,28]), cmap='gray')
                plt.show()
                
                
                
            step += 1
                

In [ ]:
training() 

In [ ]:
#run time!
z_dim = 100
LR = 0.0002
x_dim = 784
batch_size = 128
noise_var = 1.5
noise_steps = 5000

def training():
    with tf.Graph().as_default():
        sess = tf.Session()
        global_step = tf.get_variable('global_step', shape=[],dtype=tf.int32)
        
        noise = tf.train.polynomial_decay(noise_var, global_step,noise_steps,0.0,power=1.0)#linear decay to zero noise by step: noise_steps
        
        x = tf.placeholder(tf.float32, [batch_size, x_dim], 'real-images')
        
        z = tf.random_normal([batch_size, z_dim], mean=0, stddev=1.0)#Latent: Gaussian
        
        gen_x = gen(z, reuse=False)
        
        gen_x_n = gen_x + tf.random_normal([batch_size, x_dim], mean=0.0, stddev=noise)#adding noise
        x_n = x + tf.random_normal([batch_size, x_dim], mean=0.0, stddev=noise)
        
        d_fake = dis(gen_x_n, reuse=False)
        d_real = dis(x_n, reuse=True)
        
        g_train, g_loss = gen_train(d_fake, LR)
        d_train, d_loss = dis_train(d_fake, d_real, LR)
       
        initial_var = tf.global_variables_initializer()
        sess.run(initial_var)
        
        train = True
        step = 0
        
        while train == True:
            real_data = random_batch(train_images, batch_size)
            _g, _d = sess.run([g_train, d_train], feed_dict={x: real_data})
            
            if step % 100 == 0:
                loss_g, loss_d, gen_images, var_noise, gen_images_n, real_data_n = sess.run([g_loss, d_loss, gen_x, noise, gen_x_n, x_n], feed_dict={x: real_data})
                print()
                print('------------------------------------------------------------------------------')
                print('At step:', step, ', with variance of noise =', var_noise)
                print('G loss =', loss_g)
                print('D loss =', loss_d)
                print()
                print('Without Noise')
                plt.subplot(1, 4, 1)
                plt.title('real')
                plt.imshow(np.reshape(real_data[0], [28,28]), cmap='gray')
                
                plt.subplot(1, 4, 2)
                plt.title('generated')
                plt.imshow(np.reshape(gen_images[0], [28,28]), cmap='gray')
                
                plt.subplot(1, 4, 3)
                plt.title('real')
                plt.imshow(np.reshape(real_data[1], [28,28]), cmap='gray')
                
                plt.subplot(1, 4, 4)
                plt.title('generated')
                plt.imshow(np.reshape(gen_images[1], [28,28]), cmap='gray')
                plt.show()
                
                if step <= noise_steps:
                    print()
                    print('With Noise')
                    plt.subplot(1, 4, 1)
                    plt.title('real')
                    plt.imshow(np.reshape(real_data_n[0], [28,28]), cmap='gray')

                    plt.subplot(1, 4, 2)
                    plt.title('generated')
                    plt.imshow(np.reshape(gen_images_n[0], [28,28]), cmap='gray')

                    plt.subplot(1, 4, 3)
                    plt.title('real')
                    plt.imshow(np.reshape(real_data_n[1], [28,28]), cmap='gray')

                    plt.subplot(1, 4, 4)
                    plt.title('generated')
                    plt.imshow(np.reshape(gen_images_n[1], [28,28]), cmap='gray')
                    plt.show()                
                
            step += 1
                

In [ ]:
training()